In [1]:
import praw
import json
from datetime import datetime
import pytz

In [2]:
# Authenticate with Reddit
reddit = praw.Reddit(client_id = 'PO72CNBqJevVmb8HupBV_Q', 
                     client_secret = 'kCBrW2Euk37XJMIeb5ZSa6RGA6t1ZQ', 
                     user_agent = 'Analyse_it')

In [3]:
def fetch_comments(comment, comment_list):
    # Add the current comment to the list
    comment_data = {
        "body": comment.body,
        "author": comment.author.name if comment.author else None,
        "created_utc": comment.created_utc,
        "created_readable": datetime.utcfromtimestamp(comment.created_utc).strftime("%d/%m/%Y %H:%M:%S"),
        "score": comment.score,
        "replies": []
    }
    comment_list.append(comment_data)

    # Recursively retrieve replies to the current comment
    for reply in comment.replies:
        fetch_comments(reply, comment_data["replies"])

In [4]:
def fetch_reddit_data(subreddit_name, start_date, end_date):
    subreddit = reddit.subreddit(subreddit_name)

    data = []
    total_posts = 0  # Initialize the total post count
    total_comments = 0  # Initialize the total comment count

    for submission in subreddit.new(limit=None):
        created_utc = datetime.utcfromtimestamp(submission.created_utc).replace(tzinfo=pytz.utc)
        if start_date <= created_utc <= end_date:
            post_data = {
                "thread_id": f"{created_utc.timestamp()}_{submission.id}",  # Unique thread ID
                "title": submission.title,
                "self_text": submission.selftext,
                "score": submission.score,
                "author": submission.author.name if submission.author else None,
                "created_utc": submission.created_utc,
                "created_readable": created_utc.strftime("%d/%m/%Y %H:%M:%S"),
                "comments": []
            }
            total_posts += 1  # Increment the total post count
            submission.comments.replace_more(limit=None)
            for top_level_comment in submission.comments:
                fetch_comments(top_level_comment, post_data["comments"])
                total_comments += 1  # Increment the total comment count
            data.append(post_data)

    return data, total_posts, total_comments

In [5]:
# Specify the start and end dates as datetime objects with a timezone
start_date = pytz.utc.localize(datetime(2023, 9, 16))
end_date = pytz.utc.localize(datetime(2023, 10, 16, 23, 59, 59))

In [ ]:
subreddit_name = "GTAV_Mods"

data, total_posts, total_comments = fetch_reddit_data(subreddit_name, start_date, end_date)

# Store the data in a JSON file
with open("reddit_data_10.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)
print(f"Total Posts: {total_posts}")
print(f"Total Comments: {total_comments}")